In [9]:
from configparser import ConfigParser
parser = ConfigParser()
parser.read('../secrets.cfg')

portalToken = parser.get('my_api','PORTAL_TOKEN')

headers = {"Authorization": portalToken}

In [10]:
# importing the requests library
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
  
# api-endpoint
URL  = parser.get('my_api','ASH_EXAMPLE_DATA_SOURCE')

r = requests.get(url = URL, headers = headers)
    # extracting data in json format
data = r.json()

print(data)

subscriptionData=""
subscriptiondataarray= []

count = 0
if 'value' in data:
    for event in data['value']:
        count =count +1
        if count >= 19: # due to being free shit need to do: https://learn.microsoft.com/en-us/azure/search/cognitive-search-attach-cognitive-services?tabs=portal
            break
        
        if 'properties' not in event:
            print("missing props")
            continue
        
        props = set (['title','impactStartTime','impactMitigationTime','description','impact'])

        if props.issubset(event['properties'].keys()) and event['properties']['title'][0] != '_':
            s = "#################"
            s = s + "\nTitle: " + event['properties']['title'] + "\nstart time: " + event['properties']['impactStartTime'] + "\nmitigated time: " + event['properties']['impactMitigationTime'] + "\nDetails: " + re.sub('\n\n','\n',BeautifulSoup(event['properties']['description']).get_text(""))
            s = s + "\n#################\n\n"
            ##print(s)
            subscriptionData = subscriptionData + s

            subscriptiondataarray.append([event['name'], event['properties']['title'], event['properties']['impactStartTime'], event['properties']['impactMitigationTime'], re.sub('\n\n','\n',BeautifulSoup(event['properties']['description']).get_text(""))])

        
    f = open('ServiceHealthData-11764614-ffac-4e4d-8506-bdf64388ce6c.txt','w', encoding="utf-8")
    f.write(subscriptionData)

    df = pd.DataFrame(subscriptiondataarray, columns=['id','title', 'startTime', 'mitigationTime', 'Details'])

    df.to_csv('ServiceHealthData-11764614-ffac-4e4d-8506-bdf64388ce6c.csv',index=False)
    


    

{'error': {'code': 'InvalidAuthenticationToken', 'message': 'The access token is invalid.'}}
